<div>
<div style='float:left; margin-right:20pt; width:15%'><img src='img/logo-igm.png'></div>
<strong>Algorithmique et programmation 2</strong><br>
L1 Mathématiques - L1 Informatique<br>
Semestre 2
</div>

# Piles et files

## Le type *pile*

### Principe

* Conteneur avec politique « dernier arrivé, premier sorti » (LIFO)
* Utilisé partout en informatique (pile d'exécution, analyse syntaxique...)
* Type abstrait de données (défini par ses opérations)

    - ajout au sommet (souvent appelé *push*)
    - retrait du sommet (souvent appelé *pop*)
    - test si pile vide

### Implémentations

Avec des `list` :
- ajout au sommet avec `lst.append(x)`
- retrait du sommet avec `lst.pop()`
- test si vide avec `len(lst) == 0`

Complexité : tout en $O(1)$ !

Avec des listes récursives :

- ajout au sommet avec `ajout_debut(lst, x)`
- retrait du sommet avec `tete(lst)` et `suite(lst)`
- test du vide avec `est_vide(lst)`

Complexité : tout en $O(1)$ !

### Exemple d'utilisation : évaluation d'expressions arithmétiques

* Calculateur d'expressions arithmétiques parenthésées (notation « infixe »)
* Une pile pour stocker les opérandes, une pour les opérateurs

*expression utilisée dans l'exemple : $1 + (2 + 3) \times (4 \times 5)$*

In [1]:
def evaluer(expr):
    nombres = []     # pile !
    operateurs = []  # pile !
    i = 0  # curseur dans l'expression
    
    while i < len(expr):
        if expr[i] in " (":
            i += 1
            continue
        if expr[i] in "0123456789":
            nb = 0
            while expr[i] in "0123456789":
                nb = nb * 10 + int(expr[i])
                i += 1
            nombres.append(nb)
            continue
        if expr[i] == "+":
            operateurs.append(lambda a, b: a + b) 
        elif expr[i] == "*":
            operateurs.append(lambda a, b: a * b) 
        # etc.
        elif expr[i] == ")":
            op = operateurs.pop()  # on dépile un opérateur
            nbd = nombres.pop()    # puis deux opérandes
            nbg = nombres.pop()
            nombres.append(op(nbg, nbd))
        i += 1
    return nombres.pop()
    
une_exp = "(11 + ((2 + 3) * (4 + 5)))"
evaluer(une_exp)

56

Il se passe un truc rigolo si on déplace les opérateurs (sans dépasser les parenthèses) :

In [2]:
autre_exp = "(+ 11 (* (+ 2 3) (+ 4 5)))"
evaluer(autre_exp)

56

In [3]:
troisieme_exp = "(11 ((2 3 +) (4 5 +) *) +)"
print(evaluer(troisieme_exp))

56


En réalité, si l'expression est en notation polonaise inversée (postfixe) :

* On peut complètement supprimer les parenthèses
* L'évaluateur devient plus simple (pas besoin de pile d'opérateurs)
* En notation préfixe ça fonctionne aussi (en lisant à l'envers)

In [7]:
def evaluer_postfixe(expr):
    nombres = []  # pile d'opérandes !
    # pas de pile d'opérateurs !
    i = 0
    
    while i < len(expr):
        if expr[i] == " ":
            i += 1
            continue
        if expr[i] in "0123456789":
            nb = 0
            while expr[i] in "0123456789":
                nb = nb * 10 + int(expr[i])
                i += 1
            nombres.append(nb)
            continue
        
        nbd = nombres.pop()
        nbg = nombres.pop()
        if expr[i] == "+":
            nombres.append(nbg + nbd)
        elif expr[i] == "*":
            nombres.append(nbg * nbd)
        i += 1
    return nombres.pop()

exp_postfixe = "11 2 3 + 4 5 + * +"
print(evaluer_postfixe(exp_postfixe))

56


Il existe encore d'autres variantes :

* Expressions partiellement parenthésées (règles d'associativité et de priorité d'opérateurs)
* Conversion d'un format à l'autre

Thème abordé en L3 info dans les cours « analyse syntaxique » et « compilation »

## Le type « file »

### Principe

* Conteneur avec politique « premier arrivé, premier sorti » (FIFO)
* Très courant également (files d'attente de paquets TCP/IP, buffers divers...)
* Type abstrait de données (défini par ses opérations)

    - ajout en **fin** de file (souvent appelé *enqueue*)
    - retrait du **début** de file (souvent appelé *dequeue*)
    - test si file vide


### Implémentations

#### Avec des `list`

Version avec dernier arrivé en fin de liste :

- ajout en fin de file : `lst.append(x)`
- retrait en début de file : `lst.pop(0)`

**OU :**

Version avec dernier arrivé en début de liste :

- ajout en fin de file : `lst.insert(0, x)`
- retrait en début de file : `lst.pop()`


#### Avec des listes récursives

Même dilemme à résoudre... 

**Exercice :** décrire l'implémentation de chacune des opérations

### Un point sur la complexité

Problème sur les files : avec les listes Python comme avec les listes récursives, *enqueue* et *dequeue* ne peuvent pas être efficaces toutes les deux.

<table>
<tr>
<th>Opération</th>
<td>Ajout en tête</td>
<td>Retrait en tête</td>
<td>Ajout en queue</td>
<td>Retrait en queue</td>
</tr>
<tr>
<th>Type `list`</th>
<td>O(n)</td>
<td>O(n)</td>
<td>O(1)</td>
<td>O(1)</td>
</tr>
<tr>
<th>Liste récursive</th>
<td>O(1)</td>
<td>O(1)</td>
<td>O(n)</td>
<td>O(n)</td>
</tr>
</table>

Amélioration possible (un peu technique) : 
* Deux curseurs : `debut` et `fin`
* Ne pas décaler les éléments, tolérer un "trou" dans la liste
* Liste considérée comme circulaire (passage de `len(lst)-1` à `0`)
* Si la liste est pleine, redimensionner et "recaler" à 0

Il existe d'autres *structures de données* qui répondent au problème (notamment listes chaînées, vues en L2)

### Variante : file à double sens (« *deque* »)

* « deque » : *double-ended queue* (« file à deux bouts »)
* Opérations possibles : ajout et retrait efficaces en début ou en fin
* Demande encore une adaptation du modèle de liste...

In [8]:
from collections import deque
dir(deque)

['__add__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'appendleft',
 'clear',
 'copy',
 'count',
 'extend',
 'extendleft',
 'index',
 'insert',
 'maxlen',
 'pop',
 'popleft',
 'remove',
 'reverse',
 'rotate']

## Exemple final : recherche de chemin dans un graphe

Semblable au coloriage de zone (chapitre sur la récursivité)

* Avec une fonction récursive ou en stockant les cases à visiter dans une pile : recherche "en profondeur d'abord"<br>
  Avantages de la pile : *(remarque pour les pros)*
    + potentiellement moins de mémoire utilisée
    + pas de souci avec la limite de récursion
* En stockant les cases à visiter dans une file : coloriage du plus proche au plus lointain
* Sujet approfondi dans le module « Algorithmique des graphes » en L3 info (parcours de graphes)